In [ ]:
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
import math

In [ ]:
class ConvLayer:
    def __init__(self, kernel):
        self.kernel = kernel
        self.kernel_height = len(kernel)
        self.kernel_width = len(kernel[0])

    def forward(self, image):
        self.image = image
        self.output = self.convolve2d(self.image, self.kernel)
        return self.output

    def convolve2d(self, image, kernel):
        image_height = len(image)
        image_width = len(image[0])
        output_height = image_height - self.kernel_height + 1
        output_width = image_width - self.kernel_width + 1

        output = [[0.0 for _ in range(output_width)] for _ in range(output_height)]

        for i in range(output_height):
            for j in range(output_width):
                output[i][j] = sum(
                    image[i + m][j + n] * kernel[m][n]
                    for m in range(self.kernel_height)
                    for n in range(self.kernel_width)
                )

        return output

    def backward(self, d_out, learning_rate):
        d_kernel = [[0.0 for _ in range(self.kernel_width)] for _ in range(self.kernel_height)]
        d_image = [[0.0 for _ in range(len(self.image[0]))] for _ in range(len(self.image))]

        for i in range(len(d_out)):
            for j in range(len(d_out[0])):
                for m in range(self.kernel_height):
                    for n in range(self.kernel_width):
                        d_kernel[m][n] += self.image[i + m][j + n] * d_out[i][j]
                        d_image[i + m][j + n] += self.kernel[m][n] * d_out[i][j]

        self.kernel = [
            [
                self.kernel[i][j] - learning_rate * d_kernel[i][j]
                for j in range(self.kernel_width)
            ]
            for i in range(self.kernel_height)
        ]

        return d_image

In [ ]:
class ReLU:
    def forward(self, feature_map):
        self.feature_map = feature_map
        self.output = [[max(0.0, val) for val in row] for row in feature_map]
        return self.output

    def backward(self, d_out):
        d_feature_map = [
            [
                d_out[i][j] if self.feature_map[i][j] > 0 else 0.0
                for j in range(len(d_out[0]))
            ]
            for i in range(len(d_out))
        ]
        return d_feature_map

In [ ]:
class MaxPooling:
    def __init__(self, size=2, stride=2):
        self.size = size
        self.stride = stride

    def forward(self, feature_map):
        self.feature_map = feature_map
        self.output_height = (len(feature_map) - self.size) // self.stride + 1
        self.output_width = (len(feature_map[0]) - self.size) // self.stride + 1

        self.output = [
            [
                max(
                    feature_map[i * self.stride + m][j * self.stride + n]
                    for m in range(self.size)
                    for n in range(self.size)
                )
                for j in range(self.output_width)
            ]
            for i in range(self.output_height)
        ]

        return self.output

    def backward(self, d_out):
        d_feature_map = [[0.0 for _ in range(len(self.feature_map[0]))] for _ in range(len(self.feature_map))]

        for i in range(self.output_height):
            for j in range(self.output_width):
                start_i = i * self.stride
                start_j = j * self.stride
                pool = [
                    [self.feature_map[start_i + m][start_j + n] for n in range(self.size)]
                    for m in range(self.size)
                ]
                max_value = max(max(row) for row in pool)
                for m in range(self.size):
                    for n in range(self.size):
                        if self.feature_map[start_i + m][start_j + n] == max_value:
                            d_feature_map[start_i + m][start_j + n] = d_out[i][j]

        return d_feature_map

In [ ]:
class FullyConnectedLayer:
    def __init__(self, input_size, output_size):
        import random
        self.input_size = input_size
        self.output_size = output_size
        self.weights = [
            [random.uniform(-0.1, 0.1) for _ in range(input_size)]
            for _ in range(output_size)
        ]
        self.bias = [0.0 for _ in range(output_size)]

    def forward(self, inputs):
        self.inputs = inputs
        self.output = [
            sum(self.weights[i][j] * inputs[j] for j in range(self.input_size)) + self.bias[i]
            for i in range(self.output_size)
        ]
        return self.output

    def backward(self, d_out, learning_rate):
        d_weights = [
            [d_out[i] * self.inputs[j] for j in range(self.input_size)]
            for i in range(self.output_size)
        ]
        d_bias = [d_out[i] for i in range(self.output_size)]
        d_inputs = [
            sum(self.weights[i][j] * d_out[i] for i in range(self.output_size))
            for j in range(self.input_size)
        ]

        self.weights = [
            [
                self.weights[i][j] - learning_rate * d_weights[i][j]
                for j in range(self.input_size)
            ]
            for i in range(self.output_size)
        ]
        self.bias = [
            self.bias[i] - learning_rate * d_bias[i]
            for i in range(self.output_size)
        ]

        return d_inputs

In [ ]:
class Softmax:
    def forward(self, x):
        exp_x = [math.exp(val - max(x)) for val in x]
        sum_exp_x = sum(exp_x)
        self.output = [val / sum_exp_x for val in exp_x]
        return self.output

    def backward(self, d_out):
        d_inputs = [
            d_out[i] * self.output[i] * (1 - self.output[i])
            - sum(d_out[j] * self.output[j] for j in range(len(d_out)) if j != i)
            for i in range(len(d_out))
        ]
        return d_inputs

In [ ]:
class CNN:
    def __init__(self, conv_kernel, fc_input_size, fc_output_size):
        self.conv = ConvLayer(conv_kernel)
        self.relu = ReLU()
        self.pool = MaxPooling()
        self.fc = FullyConnectedLayer(fc_input_size, fc_output_size)
        self.softmax = Softmax()

    def forward(self, image):
        conv_out = self.conv.forward(image)
        relu_out = self.relu.forward(conv_out)
        pool_out = self.pool.forward(relu_out)


        flat_out = [pixel for row in pool_out for pixel in row]
        fc_out = self.fc.forward(flat_out)
        softmax_out = self.softmax.forward(fc_out)
        return softmax_out

    def backward(self, d_out, learning_rate):
        d_fc = self.fc.backward(d_out, learning_rate)


        d_pool = [
            d_fc[i * self.pool.output_width:(i + 1) * self.pool.output_width]
            for i in range(self.pool.output_height)
        ]

        d_relu = self.pool.backward(d_pool)
        d_conv = self.relu.backward(d_relu)
        self.conv.backward(d_conv, learning_rate)

    def train(self, images, labels, epochs, learning_rate):

        sample_flattened_image = [pixel for row in self.pool.forward(self.relu.forward(self.conv.forward(images[0]))) for pixel in row]
        print(f"FullyConnectedLayer.forward: sample_flattened_image length = {len(sample_flattened_image)}, input_size = {self.fc.input_size}")

        for epoch in range(epochs):
            print(f"Epoch {epoch + 1}/{epochs}")
            for image, label in zip(images, labels):
                pred = self.forward(image)
                d_out = [pred[i] - (1 if i == label else 0) for i in range(len(pred))]
                self.backward(d_out, learning_rate)

    def predict(self, image):
        return self.forward(image)

In [ ]:
def load_mnist():
    X, y = fetch_openml('mnist_784', version=1, return_X_y=True)
    if hasattr(X, 'values'):
        X = X.values.tolist()
    else:
        X = X.tolist()

    if hasattr(y, 'values'):
        y = y.values.tolist()
    else:
        y = y.tolist()

    return X, y

def preprocess_mnist(X, y):

    X = [
        [image[i * 28:(i + 1) * 28] for i in range(28)]
        for image in [[pixel / 255.0 for pixel in img] for img in X]
    ]
    y = [int(label) for label in y]
    return X, y


X, y = load_mnist()
print("Before preprocessing - X length:", len(X), "y length:", len(y))
X, y = preprocess_mnist(X, y)
print("After preprocessing - X length:", len(X), "y length:", len(y))


train_images, test_images, train_labels, test_labels = train_test_split(X, y, test_size=0.2, random_state=42)

conv_kernel = [
    [1, 0, -1],
    [1, 0, -1],
    [1, 0, -1]
]


cnn = CNN(conv_kernel, fc_input_size=13 * 13, fc_output_size=10)


cnn.train(train_images, train_labels, epochs=10, learning_rate=0.01)


correct = 0
total = len(test_images)
for image, label in zip(test_images, test_labels):
    prediction = cnn.predict(image)
    if prediction.index(max(prediction)) == label:

        correct += 1

print(f"Accuracy: {correct / total * 100}%")


/usr/local/lib/python3.10/dist-packages/sklearn/datasets/_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


Before preprocessing - X length: 70000 y length: 70000
After preprocessing - X length: 70000 y length: 70000
FullyConnectedLayer.forward: sample_flattened_image length = 169, input_size = 169
Epoch 1/10
Epoch 2/10
Epoch 3/10
Epoch 4/10
Epoch 5/10
Epoch 6/10
Epoch 7/10
Epoch 8/10
Epoch 9/10
Epoch 10/10
Accuracy: 90.86428571428571%
